# Análise de Dados Abertos


In [7]:
import numpy as np
import pandas as pd
from babel.numbers import format_compact_currency
import matplotlib.pyplot as plt
import locale

locale.setlocale(locale.LC_ALL, "pt_BR")
plt.style.use("grayscale")
excelsheet = "../data/criptoativos_dados_abertos_07082023.xls"

# Carrega a planilha excel
with pd.ExcelFile(excelsheet) as xls:
    df1 = pd.read_excel(
        xls,
        "Relatorio1",
        skiprows=9,
        decimal=",",
        parse_dates=[0],
        date_format="%B de %Y",
        usecols=[0, 3, 6, 7, 8],
    )
    df2 = pd.read_excel(xls, "Relatorio2")
    df3 = pd.read_excel(xls, "Relatório3")
    df4 = pd.read_excel(xls, "Relatorio4")


# Utilitários
def format_cols(data):
    for i in data.columns:
        if data[i].dtype == np.float64:
            data[i] = data[i].apply(
                lambda x: format_compact_currency(
                    x, currency="BRL", locale="pt_BR", fraction_digits=2
                )
            )

## Total de Valores Declarados por Ano


In [8]:
# Renderiza Relatório1
df1 = df1.rename(
    columns={
        "Unnamed: 0": "Ano",
        "Subtotal": "Exchanges no Exterior",
        "Subtotal.1": "Sem Exchanges",
        "Somente PJ": "Exchanges",
        "Unnamed: 8": "Total",
    }
)

df1_new = df1.groupby(pd.Grouper(key=df1.columns[0], freq="Y")).sum()
format_cols(df1_new)
df1_new

,Exchanges no Exterior,Sem Exchanges,Exchanges,Total
Ano,,,,
2019-12-31,"R$ 6,51 mil","R$ 7,12 mil","R$ 14,25 mil","R$ 27,89 mil"
2020-12-31,"R$ 27,73 mil","R$ 14,96 mil","R$ 49,38 mil","R$ 92,07 mil"
2021-12-31,"R$ 21,43 mil","R$ 33,2 mil","R$ 152,03 mil","R$ 206,65 mil"
2022-12-31,"R$ 14,64 mil","R$ 29,01 mil","R$ 123,04 mil","R$ 166,69 mil"
2023-12-31,"R$ 12,03 mil","R$ 14,71 mil","R$ 83,87 mil","R$ 110,6 mil"


## Quantidade de CPF/CNPJ Únicos


In [9]:
# Renderiza Relatório2

## Número de Operações, por gênero


In [10]:
# Renderiza Relatório3, parte 1

## Valor das Operações, por gênero


In [11]:
# Renderiza Relatório 3, parte 2

## Criptoativos Declarados


In [12]:
# Renderiza Relatório 4